# Cloud-native Geospatial

The cloud and geospatial data analysis go together very well.

In [ ]:
import adlfs
import planetary_computer
import ipyleaflet
import requests
import shapely.geometry
from IPython.display import IFrame

## Cloud-native Principals

The Planetary Computer implements many cloud-native concepts. Here, we'll just list some of them. Later on, we'll apply them.

1. You have *direct* access to *all* of the data
    - You have access to PBs of data
    - Data assets are hosted in the highly scalable Azure Blob Storage
    - You have direct access to the files, using plain HTTPs or Azure Blob Storage APIs. This means you can open the files using any tool that can speak HTTP
2. Cloud-native formats
    - Wherever possible, we use cloud-native / friendly file formats. We'll see examples using COG, Zarr, (geo)parquet, and COPC
    - Efficient access to metadata (don't need to read the whole file to understand its contents)
    - Efficient access to subsets of the data
3. Compute is next to the data
    - All of our files are in the West Europe Azure data region. For best performance, compute should be in that same data center.
4. Ability to scale
    - Azure makes it easy to get lots of compute


### Compute → Data

Putting the compute next to the data can be crucial for performance. Let's consider the simple task of reading the metadata from a COG file with `gdalinfo`.

Running this command from my laptop in Des Moines, IA, we spend a *lot* of time waiting:

```console
$ time gdalinfo /vsicurl/https://naipeuwest.blob.core.windows.net/naip/v002/ia/2019/ia_60cm_2019/42091/m_4209150_sw_15_060_20190828.tif > /dev/null
real    0m7.158s
user    0m0.195s
sys     0m0.032s
```

Running that from this Jupyter kernel, which is in the same Azure data center as the dataset, things look different.

In [ ]:
!time gdalinfo /vsicurl/https://naipeuwest.blob.core.windows.net/naip/v002/ia/2019/ia_60cm_2019/42091/m_4209150_sw_15_060_20190828.tif > /dev/null

So a nice 35x speedup!

## STAC

Having access to the data is great, but it's not enough. For example, how would you find all the Sentinel-2 images over Wyoming for July 2021? Consider what we'd do if we just had files in blob storage. We'll use `adlfs` to list some folders, to try to figure out the naming convention (we could also read the docs, but where's the fun in that?)

In [ ]:
%%time

search = catalog.search(
    collections=["sentinel-2-l2a"], bbox=wyoming_bbox, datetime="2021-07-01/2021-07-31"
)
items = search.get_all_items()

In [ ]:
len(items)

Even better: STAC is a standard. It isn't specific to Sentinel-2, or even remote sensing data. Landsat Collection 2 Level-2, which uses a completely different folder structure in blob storage, can be searched by just chagning the collection ID.

In [ ]:
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=wyoming_bbox,
    datetime="2021-07-01/2021-07-31",
    query={"eo:cloud_cover": {"lt": 10}},
)
%time items = search.get_all_items()
len(items)

## Data APIs

The Planetary Computer also provides a data API, based on [TiTiler](https://developmentseed.org/titiler/), which provides endpoints for some common geospatial analysis routines. This can be a nice alternative to setting up your own compute in Azure if you're doing something basic, like putting an image on a Map (or even more advanced things like mosaicing many images).

In [ ]:
import pystac_client
import shapely.geometry
import requests
import ipyleaflet

catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=[-111.0717, 41.0296, -103.9965, 45.02695],
    datetime="2021-07-01/2021-07-31",
    query={"eo:cloud_cover": {"lt": 10}},
)
%time items = search.get_all_items()
len(items)

In [ ]:
item = items[1]

(tiles_url,) = requests.get(item.assets["tilejson"].href).json()["tiles"]
center = shapely.geometry.shape(item.geometry).centroid.bounds[1::-1]

m = ipyleaflet.Map(
    center=center, controls=[ipyleaflet.FullScreenControl()],
)
m.add_layer(ipyleaflet.TileLayer(url=tiles_url))
m.scroll_wheel_zoom = True
m

Fun fact: the STAC and Data APIs power our [explorer](https://planetarycomputer.microsoft.com/explore?c=118.8189%2C37.4070&z=11.00).

### Scaling

Azure offers *many* ways to do compute. The [pangeo](https://pangeo.io/) community has done a ton of work on scalable JupyterHub deployments. This JupyterHub is deployed with [Dask Gateway](https://gateway.dask.org/)

<img src="https://gateway.dask.org/_images/architecture.svg"/>

As a user, you just need to request a cluster and scale it up.

In [ ]:
import dask_gateway

cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()
cluster

In [ ]:
cluster.scale(8)

In [ ]:
import dask.dataframe

ts = dask.dataframe.demo.make_timeseries()
df = ts.groupby("name").agg(["mean", "count", "sum"]).compute()
df.head()

In the background the Gateway service is talking to [Azure Kubernetes Service](https://docs.microsoft.com/en-us/azure/aks/) to start up scheduler and worker pods that will form our Dask Cluster. Kubernetes is in turn asking Azure for more physical nodes.

Closing your cluster will release the resources.

In [ ]:
cluster.close()

Make sure to stop your notebook kernel.